<center><b style="font-size:24px;">Information Theory Project(1) part-1</b></center>


In [567]:
import numpy as np
import pandas as pd
from functools import reduce

In [568]:
class Cell:
    def __init__(self, symbol, frequency , left = None, right = None):
        self.symbol = symbol
        self.frequency = frequency
        self.code = ''
        self.huffman_code = ''
        self.left = left
        self.right = right

    def __repr__(self):
        return f'symbol ({self.symbol}) ----- frequency {self.frequency}'

    def __radd__(self, other):
        return other + self.frequency
    
    def __lt__(self, other):
        return self.frequency < other.frequency

In [569]:
#sorting function
def sort_cells(arr):
    for i in range(len(arr)):
        for j in range(i + 1 , len(arr)):
            if arr[i].frequency <= arr[j].frequency:
                temp = arr[j]
                arr[j] = arr[i]
                arr[i] = temp
    return arr

In [570]:
with open('Test_text_file.txt', 'r') as F:
    text = F.read()

In [571]:
#total number of characters
characters ='abcdefghijklmnopqrstuvwxyz().,/- '
total =  len(text)
alphabet_dict_count = {char: 0 for char in characters }

In [572]:
print(alphabet_dict_count)

{'a': 0, 'b': 0, 'c': 0, 'd': 0, 'e': 0, 'f': 0, 'g': 0, 'h': 0, 'i': 0, 'j': 0, 'k': 0, 'l': 0, 'm': 0, 'n': 0, 'o': 0, 'p': 0, 'q': 0, 'r': 0, 's': 0, 't': 0, 'u': 0, 'v': 0, 'w': 0, 'x': 0, 'y': 0, 'z': 0, '(': 0, ')': 0, '.': 0, ',': 0, '/': 0, '-': 0, ' ': 0}


**Probabilities calculations**

In [573]:
for char in text:
    alphabet_dict_count[char] +=1

In [574]:
print(len(alphabet_dict_count))

33


In [575]:
print(alphabet_dict_count)

{'a': 101, 'b': 17, 'c': 37, 'd': 64, 'e': 146, 'f': 19, 'g': 20, 'h': 31, 'i': 85, 'j': 1, 'k': 1, 'l': 59, 'm': 30, 'n': 90, 'o': 88, 'p': 52, 'q': 1, 'r': 84, 's': 79, 't': 101, 'u': 26, 'v': 13, 'w': 11, 'x': 7, 'y': 9, 'z': 3, '(': 3, ')': 3, '.': 6, ',': 11, '/': 1, '-': 16, ' ': 189}


In [576]:
symbols_data = pd.DataFrame(alphabet_dict_count.items(), columns=['symbol' , 'frequency'])

In [577]:
symbols_data['probability'] = symbols_data['frequency'] / len(text)

In [578]:
summation = 0
for prob in symbols_data['probability']:
    summation += prob
print(summation)

0.9999999999999998


In [579]:
symbols_data['fixed-code'] = None
symbols_data['huffman-code'] = None
symbols_data['shanon-fano-code'] = None

In [580]:
symbols_data.head()

,symbol,frequency,probability,fixed-code,huffman-code,shanon-fano-code
0,a,101,0.071937,None,None,None
1,b,17,0.012108,None,None,None
2,c,37,0.026353,None,None,None
3,d,64,0.045584,None,None,None
4,e,146,0.103989,None,None,None


**Entropy calculations**

In [581]:
import math as m
entropy = 0
for prob in symbols_data['probability']:
    entropy += prob * m.log(1/prob,2)


In [582]:
print(f'The entropy H(s) = {entropy}')

The entropy H(s) = 4.25701056473807


# Fixed length code

In [583]:
number_of_bits = m.ceil(m.log(33,2))
print(f'The needed number of bits for fixed code is : {number_of_bits}')

The needed number of bits for fixed code is : 6


In [584]:
# Function to convert decimal 2 a binary number
def decimal2binary(num, length):
    binary = ""
    while num > 0:
        binary += str(num % 2)
        num  = num // 2
    if len(binary) < length:
        for i in range(length - len(binary)):
            binary += '0' 
    return binary[::-1 ]
        


In [585]:
decimal2binary(0,6)

'000000'

**Encoder**

In [586]:
fixed_codes = {}
for i in range(len(characters)):
    #fixed_codes[characters[i]] = decimal2binary(i,6)
    symbols_data.loc[symbols_data['symbol'] == characters[i], 'fixed-code'] = decimal2binary(i,6)

In [587]:
symbols_data.head()

,symbol,frequency,probability,fixed-code,huffman-code,shanon-fano-code
0,a,101,0.071937,000000,None,None
1,b,17,0.012108,000001,None,None
2,c,37,0.026353,000010,None,None
3,d,64,0.045584,000011,None,None
4,e,146,0.103989,000100,None,None


**Fixed code efficiency**

In [588]:

fixed_code_eff = (entropy / 6) * 100
print(f"the effciency of fixed code method is: {fixed_code_eff}")

the effciency of fixed code method is: 70.95017607896783


# Huffman code

In [589]:
huffman_cells =  [Cell(symbol, frequency) for symbol, frequency in alphabet_dict_count.items()]
print(huffman_cells)

[symbol (a) ----- frequency 101, symbol (b) ----- frequency 17, symbol (c) ----- frequency 37, symbol (d) ----- frequency 64, symbol (e) ----- frequency 146, symbol (f) ----- frequency 19, symbol (g) ----- frequency 20, symbol (h) ----- frequency 31, symbol (i) ----- frequency 85, symbol (j) ----- frequency 1, symbol (k) ----- frequency 1, symbol (l) ----- frequency 59, symbol (m) ----- frequency 30, symbol (n) ----- frequency 90, symbol (o) ----- frequency 88, symbol (p) ----- frequency 52, symbol (q) ----- frequency 1, symbol (r) ----- frequency 84, symbol (s) ----- frequency 79, symbol (t) ----- frequency 101, symbol (u) ----- frequency 26, symbol (v) ----- frequency 13, symbol (w) ----- frequency 11, symbol (x) ----- frequency 7, symbol (y) ----- frequency 9, symbol (z) ----- frequency 3, symbol (() ----- frequency 3, symbol ()) ----- frequency 3, symbol (.) ----- frequency 6, symbol (,) ----- frequency 11, symbol (/) ----- frequency 1, symbol (-) ----- frequency 16, symbol ( ) ---

In [590]:
huffman_cells = sort_cells(huffman_cells)
print(huffman_cells)

[symbol ( ) ----- frequency 189, symbol (e) ----- frequency 146, symbol (t) ----- frequency 101, symbol (a) ----- frequency 101, symbol (n) ----- frequency 90, symbol (o) ----- frequency 88, symbol (i) ----- frequency 85, symbol (r) ----- frequency 84, symbol (s) ----- frequency 79, symbol (d) ----- frequency 64, symbol (l) ----- frequency 59, symbol (p) ----- frequency 52, symbol (c) ----- frequency 37, symbol (h) ----- frequency 31, symbol (m) ----- frequency 30, symbol (u) ----- frequency 26, symbol (g) ----- frequency 20, symbol (f) ----- frequency 19, symbol (b) ----- frequency 17, symbol (-) ----- frequency 16, symbol (v) ----- frequency 13, symbol (,) ----- frequency 11, symbol (w) ----- frequency 11, symbol (y) ----- frequency 9, symbol (x) ----- frequency 7, symbol (.) ----- frequency 6, symbol ()) ----- frequency 3, symbol (() ----- frequency 3, symbol (z) ----- frequency 3, symbol (/) ----- frequency 1, symbol (q) ----- frequency 1, symbol (k) ----- frequency 1, symbol (j) -

**Encoding symbols**

In [591]:
import heapq as h
h.heapify(huffman_cells)

In [592]:
while len(huffman_cells) != 1:
    left_cell = h.heappop(huffman_cells)
    right_cell = h.heappop(huffman_cells)
    
    left_cell.huffman_code += "0"
    right_cell.huffman_code += "1"
    
    nNode = Cell(left_cell.symbol + right_cell.symbol, left_cell.frequency + right_cell.frequency, left_cell, right_cell) 
    h.heappush(huffman_cells, nNode)

In [593]:
def save_codes(node, val=''): 
  
    # huffman code for current node 
    newVal = val + str(node.huffman_code) 
  
    # if node is not an edge node 
    # then traverse inside it 
    if(node.left): 
        save_codes(node.left, newVal) 
    if(node.right): 
        save_codes(node.right, newVal) 
  
        # if node is edge node then 
        # display its huffman code 
    if(not node.left and not node.right): 
        symbols_data.loc[symbols_data['symbol'] == node.symbol , 'huffman-code'] = newVal
save_codes(huffman_cells[0])

In [594]:
symbols_data.head()

,symbol,frequency,probability,fixed-code,huffman-code,shanon-fano-code
0,a,101,0.071937,000000,1101,None
1,b,17,0.012108,000001,000101,None
2,c,37,0.026353,000010,00011,None
3,d,64,0.045584,000011,0000,None
4,e,146,0.103989,000100,001,None


In [595]:
l_avg_huffman = 0
for i in range(len(symbols_data)):
    l_avg_huffman += len(symbols_data.iloc[i]['huffman-code']) * symbols_data.iloc[i]['probability']
print(f'The average code length for huffman code is {l_avg_huffman}')

The average code length for huffman code is 4.276353276353276


In [596]:
print(f'the efficency of huffman code is {(entropy/l_avg_huffman)*100} %')

the efficency of huffman code is 99.5476820934752 %


**Coding the whole text file**

In [597]:
def huffman_encoder(text):
    code = ''
    for char in text:
        char_code = symbols_data.loc[symbols_data['symbol'] == char, 'huffman-code'].item()
        code += char_code
    return code 

In [598]:
huffman_file_code = ''
for char in text:
    char_code = symbols_data.loc[symbols_data['symbol'] == char, 'huffman-code'].item()
    huffman_file_code += char_code
with open('huffman_code.txt', 'w') as f:
    f.write(huffman_file_code)

In [599]:
huffman_text = ''
code = ''
with open('huffman_code.txt', 'r') as f:
    huffman_txt_code = f.read() 
for bit in huffman_txt_code:
    code += bit
    if code in symbols_data['huffman-code'].values:
         symbol =  symbols_data.loc[symbols_data['huffman-code'] == code]['symbol'].item()
         huffman_text += symbol
         code = ''
print(huffman_text)

in this paper, a novel decorrelation-based concurrent digital predistortion (dpd) solution is proposed for dual-band transmitters (tx) employing a single wideband power amplifier (pa), and utilizing just a single feedback receiver path. the proposed decorrelation-based parameter learning solution is both flexible and simple, and operates in a closed-loop manner, opposed to the widely applied indirect learning architecture. the proposed decorrelation-based learning and dpd processing can also be effectively applied to more ordinary single-band transmissions, as well as generalized to more than two transmit bands. through a comprehensive analysis covering both the dpd parameter learning and the main path processing, it is shown that the complexity of the proposed concurrent dpd is substantially lower compared with the other state-of-the-art concurrent dpd methods. extensive set of quantitative simulation and rf measurement results are also presented, using a base-station pa as well as a 

# Shanon-Fano code

In [600]:
shanon_codes = {char : "" for char in characters}
print(shanon_codes)

{'a': '', 'b': '', 'c': '', 'd': '', 'e': '', 'f': '', 'g': '', 'h': '', 'i': '', 'j': '', 'k': '', 'l': '', 'm': '', 'n': '', 'o': '', 'p': '', 'q': '', 'r': '', 's': '', 't': '', 'u': '', 'v': '', 'w': '', 'x': '', 'y': '', 'z': '', '(': '', ')': '', '.': '', ',': '', '/': '', '-': '', ' ': ''}


**Encoder**

In [601]:
c1 = Cell('A', 0.5)
print(c1)

symbol (A) ----- frequency 0.5


In [602]:
shanon_cells = [Cell(char,frequency) for char , frequency in alphabet_dict_count.items()]
print(shanon_cells) 

[symbol (a) ----- frequency 101, symbol (b) ----- frequency 17, symbol (c) ----- frequency 37, symbol (d) ----- frequency 64, symbol (e) ----- frequency 146, symbol (f) ----- frequency 19, symbol (g) ----- frequency 20, symbol (h) ----- frequency 31, symbol (i) ----- frequency 85, symbol (j) ----- frequency 1, symbol (k) ----- frequency 1, symbol (l) ----- frequency 59, symbol (m) ----- frequency 30, symbol (n) ----- frequency 90, symbol (o) ----- frequency 88, symbol (p) ----- frequency 52, symbol (q) ----- frequency 1, symbol (r) ----- frequency 84, symbol (s) ----- frequency 79, symbol (t) ----- frequency 101, symbol (u) ----- frequency 26, symbol (v) ----- frequency 13, symbol (w) ----- frequency 11, symbol (x) ----- frequency 7, symbol (y) ----- frequency 9, symbol (z) ----- frequency 3, symbol (() ----- frequency 3, symbol ()) ----- frequency 3, symbol (.) ----- frequency 6, symbol (,) ----- frequency 11, symbol (/) ----- frequency 1, symbol (-) ----- frequency 16, symbol ( ) ---

In [603]:
#sorting the symbols according to probabilties 
shanon_cells = sort_cells(shanon_cells)

In [604]:
print(shanon_cells)

[symbol ( ) ----- frequency 189, symbol (e) ----- frequency 146, symbol (t) ----- frequency 101, symbol (a) ----- frequency 101, symbol (n) ----- frequency 90, symbol (o) ----- frequency 88, symbol (i) ----- frequency 85, symbol (r) ----- frequency 84, symbol (s) ----- frequency 79, symbol (d) ----- frequency 64, symbol (l) ----- frequency 59, symbol (p) ----- frequency 52, symbol (c) ----- frequency 37, symbol (h) ----- frequency 31, symbol (m) ----- frequency 30, symbol (u) ----- frequency 26, symbol (g) ----- frequency 20, symbol (f) ----- frequency 19, symbol (b) ----- frequency 17, symbol (-) ----- frequency 16, symbol (v) ----- frequency 13, symbol (,) ----- frequency 11, symbol (w) ----- frequency 11, symbol (y) ----- frequency 9, symbol (x) ----- frequency 7, symbol (.) ----- frequency 6, symbol ()) ----- frequency 3, symbol (() ----- frequency 3, symbol (z) ----- frequency 3, symbol (/) ----- frequency 1, symbol (q) ----- frequency 1, symbol (k) ----- frequency 1, symbol (j) -

In [605]:
def shanon_encoder(cells):
    if len(cells) <= 1:
        return
    total_frequency = sum(cells)
    half_frequency = total_frequency / 2
    cumulative_frequency = 0
    split_index = -1
    min_difference = 9999999999999999999999999999999999999
    for i, freq in enumerate(cell.frequency for cell in cells):
        cumulative_frequency += freq
        difference = abs(cumulative_frequency - half_frequency)

        if difference < min_difference:
            min_difference = difference
            split_index = i
    left_cells = cells[ : split_index + 1]
    right_cells = cells[split_index + 1 : ]
    for cell in left_cells:
        cell.code += '0'
    for cell in right_cells:
        cell.code += '1'
    
    shanon_encoder(left_cells)
    shanon_encoder(right_cells)

In [606]:
shanon_encoder(shanon_cells)

In [607]:
for cell in shanon_cells:
    symbols_data.loc[symbols_data['symbol'] == cell.symbol, 'shanon-fano-code'] = cell.code

In [608]:
symbols_data.sort_values(by='frequency', ascending=False , inplace= True)

In [609]:
symbols_data = symbols_data.reset_index(drop=True)

In [610]:
symbols_data.head()

,symbol,frequency,probability,fixed-code,huffman-code,shanon-fano-code
0,,189,0.134615,100000,101,000
1,e,146,0.103989,000100,001,001
2,a,101,0.071937,000000,1101,0101
3,t,101,0.071937,010011,1100,0100
4,n,90,0.064103,001101,1001,0110


In [611]:
l_avg_shannon = 0
for i in range(len(symbols_data)):
    l_avg_shannon += symbols_data['probability'].iloc[i] * len(symbols_data['shanon-fano-code'].iloc[i])
print(f'the average code length for shanon fano code is {l_avg_shannon}')
print(f'the effciency of shanon fano encoder is {(entropy / l_avg_shannon)*100}')

the average code length for shanon fano code is 4.278490028490028
the effciency of shanon fano encoder is 99.4979662542409


In [612]:
symbols_data

,symbol,frequency,probability,fixed-code,huffman-code,shanon-fano-code
0,,189,0.134615,100000,101,000
1,e,146,0.103989,000100,001,001
2,a,101,0.071937,000000,1101,0101
3,t,101,0.071937,010011,1100,0100
4,n,90,0.064103,001101,1001,0110
5,o,88,0.062678,001110,1000,0111
6,i,85,0.060541,001000,0111,1000
7,r,84,0.059829,010001,0110,1001
8,s,79,0.056268,010010,0100,1010
9,d,64,0.045584,000011,0000,10110


**Coding the whole file**

In [613]:
def shanon_fano_encoder(text):
    file_code = ''
    for char in text:
        file_code += symbols_data.loc[symbols_data['symbol'] == char, 'shanon-fano-code'].item()
    return file_code

In [614]:
file_code = ''
for char in text:
    file_code += symbols_data.loc[symbols_data['symbol'] == char, 'shanon-fano-code'].item()

In [615]:
file_code

'100001100000100110101000101000011000010111000001100111110110000101000011001111111010001101110001011000111001011110011001001101110101010010000111011011110011101101011010001101100001100101110110110011110001001100100101100100000101101000111001100001000101101110001100010010011011010001010010001111001010010000111011000011111110110110110001011011111110000010100111101111110000100100001110110000100010100001100010010111110000111101000110110000111010011110010001011011100001011011111110011101101010110101100000100100101010110101011011100001000100001100110100001111111010100111111001111111000000011101111000101110111111110110000110111001000010100010101000011011100110111001000111110010001011000111101101010110101100001100001111111100001100100001011101111000101111000111010100000110010001111111011100001011111111001111011000010101101011000011100001001000101111000111111110100001101110010001111111111111100010100100000010100010101000011011100110111001000111010001001101101110110101110011111111111000010010011

In [616]:
with open('shanon_code.txt','w') as f:
    f.write(file_code)

**Decoding**

In [617]:
with open('shanon_code.txt', 'r') as f:
    code = f.read()

In [618]:
symbol_code = ''
shanon_text = ''
for bit in code:
    symbol_code += bit
    if symbol_code in symbols_data['shanon-fano-code'].values:
        symbol =  symbols_data.loc[symbols_data['shanon-fano-code'] == symbol_code]['symbol'].item()
        shanon_text += symbol
        symbol_code = ''

In [619]:
shanon_text

'in this paper, a novel decorrelation-based concurrent digital predistortion (dpd) solution is proposed for dual-band transmitters (tx) employing a single wideband power amplifier (pa), and utilizing just a single feedback receiver path. the proposed decorrelation-based parameter learning solution is both flexible and simple, and operates in a closed-loop manner, opposed to the widely applied indirect learning architecture. the proposed decorrelation-based learning and dpd processing can also be effectively applied to more ordinary single-band transmissions, as well as generalized to more than two transmit bands. through a comprehensive analysis covering both the dpd parameter learning and the main path processing, it is shown that the complexity of the proposed concurrent dpd is substantially lower compared with the other state-of-the-art concurrent dpd methods. extensive set of quantitative simulation and rf measurement results are also presented, using a base-station pa as well as a

In [620]:
symbols_data.head()

,symbol,frequency,probability,fixed-code,huffman-code,shanon-fano-code
0,,189,0.134615,100000,101,000
1,e,146,0.103989,000100,001,001
2,a,101,0.071937,000000,1101,0101
3,t,101,0.071937,010011,1100,0100
4,n,90,0.064103,001101,1001,0110


<center><b style="font-size:24px;">Information Theory Project(1) part-2</b></center>

# Channel code

In [621]:
def awgn(signal, snr_dB):
    """
    Add Additive White Gaussian Noise (AWGN) to a signal.

    Parameters:
    signal (numpy array): The input signal.
    snr_dB (float): Desired Signal-to-Noise Ratio in dB.

    Returns:
    numpy array: Signal with added AWGN.
    """
    # Calculate signal power
    signal_power = np.mean(np.abs(signal) ** 2)

    # Convert SNR from dB to linear scale
    snr_linear = 10 ** (snr_dB / 10)

    # Calculate noise power
    noise_power = signal_power / snr_linear

    # Generate Gaussian noise
    noise = np.sqrt(noise_power) * np.random.normal(size=signal.shape)

    # Add noise to the signal
    noisy_signal = signal + noise
    return noisy_signal

In [622]:
def channel_encoder(binary_stream):
    num_of_chuncks = np.ceil(len(binary_stream) / 4)
    
    code = ''
    
    for i in range(int(num_of_chuncks)):
        start = 4 * i
        end = start + 4
        chunck = binary_stream[start: end]
        bit_list = list(chunck)
        
        bit_list = [int(bit) for bit in bit_list]
        b0 = reduce(lambda x, y: x ^ y, [bit_list[i] for i in [0, 2, 3]])
        b1 = reduce(lambda x, y: x ^ y, [bit_list[i] for i in [0, 1, 2]])
        b2 = reduce(lambda x, y: x ^ y, [bit_list[i] for i in [1, 2, 3]])
        chunck = chunck + str(b0) + str(b1) + str(b2)
        code += chunck
    binary_code = list(map(int, code))
    return np.asarray(binary_code)

In [623]:
def channel_decoder(Rx):
    binary_Rx = (Rx >= 0.5).astype(int)
    #decoded_bits = np.zeros(np.size(binary_Rx))
    recieved_bits = binary_Rx.reshape(-1,7)
    symbols, _ = recieved_bits.shape
    decoded_bits = np.zeros(4 * symbols)
    
    for i in range(symbols):
        start_idx = 4 * i
        end_idx = start_idx + 4
        symbol_code = recieved_bits[i,:]
        s0 = reduce(lambda x, y: x ^ y, [symbol_code[i] for i in [4, 0, 2, 3]])
        s1 = reduce(lambda x, y: x ^ y, [symbol_code[i] for i in [5, 0, 1, 2]])
        s2 = reduce(lambda x, y: x ^ y, [symbol_code[i] for i in [6, 1, 2, 3]])
        syndrome = s0 * 1 + s1 * 2 + s2 * 4
        if syndrome == 0:
            decoded_bits[start_idx: end_idx] = symbol_code[0:4]
        elif syndrome == 1:
            symbol_code[4] = symbol_code[4] ^ 1
            decoded_bits[start_idx: end_idx] = symbol_code[0:4]
        elif syndrome == 2:
            symbol_code[5] = symbol_code[5] ^ 1
            decoded_bits[start_idx: end_idx] = symbol_code[0:4]
        elif syndrome == 3:
            symbol_code[0] = symbol_code[0] ^ 1
            decoded_bits[start_idx: end_idx] = symbol_code[0:4]
        elif syndrome == 4:
            symbol_code[6] = symbol_code[6] ^ 1
            decoded_bits[start_idx: end_idx] = symbol_code[0:4]
        elif syndrome == 5:
            symbol_code[3] = symbol_code[3] ^ 1
            decoded_bits[start_idx: end_idx] = symbol_code[0:4]
        elif syndrome == 6:
            symbol_code[1] = symbol_code[1] ^ 1
            decoded_bits[start_idx: end_idx] = symbol_code[0:4]
        elif syndrome == 7:
            symbol_code[2] = symbol_code[2] ^ 1
            decoded_bits[start_idx: end_idx] = symbol_code[0:4]
    return decoded_bits

In [624]:
def source_decoder(bit_stream):
    recieved_bits = (bit_stream >= 0.5).astype(int)
    return recieved_bits

# Program

In [627]:
with open('Test_text_file.txt', 'r') as F:
    text = F.read()
print('choose compression type \n 1-Shannon Fano Code \n 2-Huffman Code')
options = [1,2]
compression_type = int(input('choose options from 1, 2'))
while(compression_type not in options):
    compression_type = int(input('error please choose a number from the provided list 1, 2'))
code = None
match compression_type:        
    case 1:
        compression = 'shanon-fano-code'
        code = shanon_fano_encoder(text)
        if(len(code) % 4 != 0):
            extra_bits = 4 - (len(code) % 4)
            code += extra_bits * '0'
        
        transmitted_code = list(code)
        transmitted_code = np.asarray([int(bit) for bit in transmitted_code ])
    case 2:
        compression = 'huffman-code'
        code = huffman_encoder(text)
        if(len(code) % 4 != 0):
            extra_bits = 4 - (len(code) % 4)
            code += extra_bits * '0'
        transmitted_code = list(code)
        transmitted_code = np.asarray([int(bit) for bit in transmitted_code])
Tx_channel_code = channel_encoder(code)

choose compression type 
 1-Shannon Fano Code 
 2-Huffman Code


**Adding Noise**

In [637]:
SNRs = [0, 5, 10, 15]
for snr in SNRs:
    Rx_channel_code = awgn(Tx_channel_code, snr)
    Rx_source_code = awgn(transmitted_code, snr)
    #print(len(Rx_channel_code), len(Rx_source_code))
    Rx_channel_decoded = channel_decoder(Rx_channel_code)
    Rx_source_decoded = source_decoder(Rx_source_code)

    BER_CC = (np.sum(transmitted_code != Rx_channel_decoded)) / len(transmitted_code)
    BER_SC = (np.sum(transmitted_code != Rx_source_decoded)) / len(transmitted_code)
    print(f'At SNR = {snr}')
    print(f'The BER in channel code is {round(BER_CC,5)} \n the BER in source code is {round(BER_SC,5)}')
    print(100 * '-')

At SNR = 0
The BER in channel code is 0.24018 
 the BER in source code is 0.24035
----------------------------------------------------------------------------------------------------
At SNR = 5
The BER in channel code is 0.06758 
 the BER in source code is 0.10386
----------------------------------------------------------------------------------------------------
At SNR = 10
The BER in channel code is 0.00133 
 the BER in source code is 0.01248
----------------------------------------------------------------------------------------------------
At SNR = 15
The BER in channel code is 0.0 
 the BER in source code is 0.0
----------------------------------------------------------------------------------------------------
